In [ ]:
import os
import json

#LangChain imports
from langchain import PromptTemplate, LLMChain
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
%pip install langchain-community
%pip install -U langchain-openai


In [ ]:

#4o the best model available for us
llm = ChatOpenAI(
    model_name="gpt-4",
    temperature=0  #For consistent, fact-based responses
)


In [ ]:
#this is used to store our entire article, if we want more articles follow this
#however if we have an article with more then 8k characters, chat 4o has a constraint of 8k characters so we will need to split
#into chunks if needed
article_text = """
Members of the Insurance and Real Estate Committee introduced a broad bill aimed at providing people with long-term care insurance some relief from large rate hikes and boosting transparency around the state’s rate review process.

But policyholders who gathered at a public hearing Tuesday called on leaders of the committee to offer consumers more protection while lobbyists for the insurance sector opposed the measure.

The proposal would limit premium increases for certain long-term care insurance policies, create a tax credit for consumers with these plans, require prospective buyers to be notified about the risk of rate increases, and mandate that a public hearing be held when rate hike requests exceed 10%, among other changes.

“If a reduction in these outrageous premiums is not legislated, policyholders will be forced to drop these policies,” said Jan Kritzman, who herself has endured large rate increases. “If they end up in a nursing home, then here comes Title 19 Medicaid. Thank you, taxpayers of the state of Connecticut.

“My husband and I have long-term health insurance policies from Transamerica. We have endured a 350% increase in our premium since 2004. This May, our annual premium combined will be more than $6,000 … this is after I pared down the policy benefits in 2021.”

Under the bill, insurers would not be able to impose rate increases that exceed the most recent calendar year average in the consumer price index, provided the policy was purchased on or before Dec. 31, 1985 and the policyholder has reached age 80 or has paid a maximum lifetime premium hike of at least 400%. Policyholders questioned how many people would qualify for the benefit and encouraged the committee to expand protections.

“They sent me a letter that said, ‘We are going to increase your rates approved by the state from $2,900 and change — almost $3,000 — to in two years, it would be going up to $6,000,’” said Kenneth Kollmeyer of Farmington, describing a letter he received from his insurer. “The two of us [Kollmeyer and his wife] would have to pay $12,000. Our social security, our fixed income, wasn’t going up at that rate.” Policyholders said insurance providers have offered them buyouts to try to get them to drop their plans, sometimes offering more than the amount they paid in premiums over the years.

“The benefit to the insurance company is that for every dollar they pay in a policy buyback, they are removing sometimes 10 to 20 times that amount of liability from their balance sheet,” Newington resident David Schwartzer said.

Schwartzer asked insurance committee members to do more to hold carriers accountable for their actions that “created this crisis in the first place.”

“Residents, mostly seniors … are being forced to choose between buying food, paying their rent or mortgage or paying for their long-term care insurance,” he said. “Solving this problem is not easy. It takes courage to write new laws that will serve the citizens of Connecticut and not the insurance industry. Does this committee have the courage to do that? I believe you do.”

Lobbyists for the insurance industry expressed opposition to the measure.

“While this bill is well-intentioned, it would have serious consequences for both consumers and insurers by undermining the financial stability of long-term care insurance,” said Brooke Foley, general counsel for the Insurance Association of Connecticut. “The Connecticut Insurance Department already provides robust oversight of long-term care insurance.”

“As insurers, we have a fundamental obligation to ensure the financial sustainability of the products we provide to consumers. Actuarially sound rates are essential to achieving this goal, and deviating from this principle may lead to inadequate premiums, jeopardizing insurer stability and consumer protection,” added Amanda Herrington, executive director of AHIP [America’s Health Insurance Plans], in written testimony with two other groups. Rep. Kerry Wood, D-Rocky Hill, a co-chair of the insurance committee, questioned how the legislation would create solvency issues for companies that are posting profits of $10 million or $20 million.

“It’s going to impact different insurance companies differently,” Jan Graeber, senior actuary for the American Council of Life Insurers, replied. “If it’s a company that only sells long-term care [policies] and they don’t have other assets to fund deficiencies in the long-term care block, they’re going to be impacted differently than a company that has several lines of business.”

The Connecticut Mirror reported last month that the annual cost of maintaining long-term care coverage has skyrocketed for many residents due to miscalculations by insurers on how long people would live, the price of care and how many would need it. Policyholders complain of dramatic rate increases, often exceeding 50% and, for a few dozen people, as high as 174%, according to a CT Mirror analysis.

Nearly 100,000 people in Connecticut have long-term care insurance, coverage that, depending on the policy, supports skilled in-home care, rehabilitation therapy, assisted living, nursing home stays and respite care.

A CT Mirror review of rate hikes from January 2019 to October 2024 shows more than 17,000 people with long-term care policies have gotten hit with increases of 50% or more. Some of the biggest companies in the market, including Genworth Financial, Metropolitan Life Insurance Company and Transamerica Life Insurance Company, requested hikes for five years in a row beginning in 2019.

When large providers seek premium increases, thousands of consumers can be impacted. In 2019, for example, Genworth Financial requested a 40% rate hike on more than 9,000 Connecticut policyholders. In 2021, Transamerica requested a 20% rate increase on 8,000 policies. The Connecticut Insurance Department approved both requests with no changes.

In 2022, Genworth raised rates for more than 2,000 people by an average of 97%, with increases ranging from 79% to 173%, depending on the policy. The approved amounts were a slight reduction from the company’s original request.

As consumers are squeezed, grievances filed with the Insurance Department have mounted. The office received more than 700 complaints in the last six years about long-term care insurance, mostly regarding rising premiums.

Solutions have been elusive. Legislators introduced more than 50 bills from 2019 to 2024, with only a small number passing and offering limited relief. They blamed budget constraints and pushback from insurers, among other problems.

So far this year, at least 13 bills have been raised to tackle the issue, including the insurance committee’s proposal.

Rep. Matt Blumenthal, D-Stamford, House chair of the Government Administration and Elections Committee, urged the committee to address the soaring cost of long-term care plans.

“There is a crisis with regard to long-term care insurance and affordability in this state, with thousands of individuals being priced out of their long-term care insurance after having paid into it for decades and now being forced to drop benefits and not being able to take advantage of the benefits they paid into for so long,” he said.

Blumenthal has submitted his own bill on the topic, and some policyholders encouraged insurance committee members to incorporate elements of it into their legislation.

Wood said the committee is still working on the bill and would incorporate suggestions raised by lawmakers and policyholders on Tuesday.

“We are making this a priority this legislative session,” she said.
"""

In [ ]:
template_str = """
You are a helpful assistant analyzing an article about insurance-related issues in Connecticut.
We are working on a project titled:
'Insurance Complaints in Connecticut: A Data-Driven Approach to Consumer Trust.'

Our overarching goal:
- Analyze 65,000 consumer complaints filed against insurance companies in Connecticut
- Identify which insurers are most responsive
- Examine complaint frequencies, resolution times, financial recovery amounts, and sentiment
- Guide Connecticut residents toward more trustworthy insurance providers
- Add valuable unstructured data to our existing datasets

Given the ARTICLE TEXT below, extract the following fields in valid JSON format, focusing on information relevant to insurance complaints, policyholder concerns, insurer practices, or legislation:

{{
  "hearing_date": "",            // If provided (e.g., "Feb 11, 2025"), else blank
  "location": "",                // Where the hearing or event took place
  "insurers_mentioned": [],      // Any insurer names, companies, or associations mentioned
  "policyholder_issues": [],     // Key complaints, grievances, or issues raised by policyholders
  "legislative_proposals": [],   // Legislative or policy measures suggested or introduced
  "sentiment_overall": "",       // Brief description (e.g., 'critical', 'supportive', 'mixed')
  "notable_quotes": [],          // List of direct quotations from the text (short excerpts)
  "data_insights_for_project": ""
    // Summarize how this article's content could be integrated
    // or cross-referenced with our 65K insurance complaints database
    // for deeper analysis or context
}}

Rules for extraction:
- Return ONLY the JSON object.
- If a field is not discussed, leave it blank or empty.
- Focus on tying the content back to how it might inform or enrich
  our broader CT insurance complaint analysis.

ARTICLE TEXT:
{chunk}
"""


prompt = PromptTemplate(
    template=template_str,
    input_variables=["chunk"]
)

In [ ]:
#this may change depending on if we split article into chunks, if we do split into chunks for larger article
#we will need another code block for article chunk splitting for tokenization

chain = LLMChain(llm=llm, prompt=prompt)
chunks = [article_text]
structured_responses = []

for i, chunk in enumerate(chunks):
    print(f"\n--- Processing Chunk {i+1}/{len(chunks)} ---")
    response = chain.run(chunk=chunk)
    print("Raw LLM Response:\n", response)

    #Attempt JSON parse
    try:
        data = json.loads(response)
    except json.JSONDecodeError:
        #In case of JSON parse issues, store raw text
        data = {"error": "Could not parse JSON", "raw_response": response}

    structured_responses.append(data)

print("\nAll Structured Responses:\n")
for sr in structured_responses:
    print(json.dumps(sr, indent=2))




--- Processing Chunk 1/1 ---
Raw LLM Response:
 {
  "hearing_date": "Tuesday",
  "location": "Connecticut",
  "insurers_mentioned": ["Transamerica", "Genworth Financial", "Metropolitan Life Insurance Company", "Transamerica Life Insurance Company"],
  "policyholder_issues": ["Large rate hikes", "Lack of transparency in rate review process", "Risk of dropping policies due to high premiums", "Inadequate protection from legislation", "Buyouts from insurance providers", "Dramatic rate increases"],
  "legislative_proposals": ["Limit premium increases for certain long-term care insurance policies", "Create a tax credit for consumers with these plans", "Require prospective buyers to be notified about the risk of rate increases", "Mandate a public hearing when rate hike requests exceed 10%"],
  "sentiment_overall": "Mixed",
  "notable_quotes": ["“If a reduction in these outrageous premiums is not legislated, policyholders will be forced to drop these policies,” said Jan Kritzman.", "“Resident